In [1]:
import requests
import json
from datetime import datetime,timedelta
import pandas as pd
import numpy as np
import datetime as dt

# B. Khám phá dữ liệu

## 1. Đọc dữ liệu, tính số dòng và số cột

Đọc dữ liệu từ file `weather-2021.csv` và in ra 5 dòng đầu tiên

In [2]:
df = pd.read_csv('weather-2021.csv')
df.head()

,key,class,expire_time_gmt,obs_id,obs_name,valid_time_gmt,day_ind,temp,wx_icon,icon_extd,...,clds,water_temp,primary_wave_period,primary_wave_height,primary_swell_period,primary_swell_height,primary_swell_direction,secondary_swell_period,secondary_swell_height,secondary_swell_direction
0,VVTS,observation,1609441200,VVTS,Ho Chi Minh City,1609434000,N,75,29,2900,...,SCT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,VVTS,observation,1609443000,VVTS,Ho Chi Minh City,1609435800,N,75,29,2900,...,SCT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,VVTS,observation,1609444800,VVTS,Ho Chi Minh City,1609437600,N,75,29,2900,...,SCT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,VVTS,observation,1609446600,VVTS,Ho Chi Minh City,1609439400,N,73,29,2900,...,SCT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,VVTS,observation,1609448400,VVTS,Ho Chi Minh City,1609441200,N,73,29,2900,...,SCT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Tính số dòng, số cột và lưu vào 2 biến `num_rows` và `num_cols`

In [3]:
num_rows, num_cols = df.shape
print(f'Number of rows: {num_rows}\nNumber of columns: {num_cols}')

Number of rows: 17390
Number of columns: 45


## 2. Mỗi dòng có ý nghĩa gì? Có vấn đề các dòng có ý nghĩa khác nhau không?

Mỗi dòng trong tập dữ liệu là thông tin và các chỉ số về thời tiết của quận Tân Bình (TP.HCM) tại thời điểm nhất định (cập nhật 30 phút/lần). Có vẻ không có vấn đề các dòng có ý nghĩa khác nhau, tức không có dòng nào bị 'lạc loài'

## 3. Dữ liệu có các dòng bị lặp không?

Ta kiểm tra xem có dòng nào bị lặp không bằng cách sử dụng các phương thức `duplicated()` và `any()` và lưu kết quả vào biến `have_duplicated_rows`. Biến này sẽ có giá trị True nếu dữ liệu có các dòng bị lặp và có giá trị False nếu không có dòng bị lặp

In [4]:
have_duplicate_rows = df.duplicated().any()
have_duplicate_rows

False

Như vậy, dữ liệu không có dòng nào bị lặp lặp 

## 4. Tỉ lệ giá trị thiếu của từng cột

Ta tính tỉ lệ giá trị thiếu của từng cột. Đầu tiên ta dùng `isnull()` để biết được các giá trị thiếu, sau đó dùng `sum()` để tính tổng số giá trị thiếu theo từng cột. Cuối cùng ta chia cho tổng số dòng để tính tỉ lệ giá trị thiếu của mỗi cột và lưu kết quả vào biến `missing_ratio`

In [5]:
missing_ratio = df.isnull().sum()
missing_ratio = missing_ratio / num_rows
missing_ratio

key                          0.000000
class                        0.000000
expire_time_gmt              0.000000
obs_id                       0.000000
obs_name                     0.000000
valid_time_gmt               0.000000
day_ind                      0.000000
temp                         0.000000
wx_icon                      0.000000
icon_extd                    0.000000
wx_phrase                    0.000000
pressure_tend                1.000000
pressure_desc                1.000000
dewPt                        0.000115
heat_index                   0.000115
rh                           0.000115
pressure                     0.000575
vis                          0.000000
wc                           0.000000
wdir                         0.255952
wdir_cardinal                0.000000
gust                         0.994767
wspd                         0.000000
max_temp                     1.000000
min_temp                     1.000000
precip_total                 1.000000
precip_hrly 

In [6]:
df.describe()

,expire_time_gmt,valid_time_gmt,temp,wx_icon,icon_extd,pressure_tend,pressure_desc,dewPt,heat_index,rh,...,terse_phrase,water_temp,primary_wave_period,primary_wave_height,primary_swell_period,primary_swell_height,primary_swell_direction,secondary_swell_period,secondary_swell_height,secondary_swell_direction
count,1.739000e+04,1.739000e+04,17390.000000,17390.000000,17390.000000,0.0,0.0,17388.000000,17388.000000,17388.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
mean,1.625190e+09,1.625183e+09,82.682691,28.541116,2853.851466,NaN,NaN,74.017483,88.648781,77.013055,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,9.068949e+06,9.068949e+06,5.317355,5.450935,532.476455,NaN,NaN,4.949708,8.770057,16.298229,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.609441e+09,1.609434e+09,64.000000,4.000000,400.000000,NaN,NaN,46.000000,64.000000,21.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,1.617349e+09,1.617342e+09,79.000000,28.000000,2800.000000,NaN,NaN,72.000000,82.000000,66.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,1.625201e+09,1.625194e+09,82.000000,29.000000,2900.000000,NaN,NaN,75.000000,88.000000,79.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,1.633037e+09,1.633030e+09,86.000000,30.000000,3000.000000,NaN,NaN,77.000000,95.000000,89.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
max,1.640889e+09,1.640882e+09,99.000000,47.000000,3849.000000,NaN,NaN,84.000000,114.000000,100.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Loại bỏ một số cột không cần thiết

Ta có thể thấy có rất nhiều cột không có giá trị (hay tỉ lệ dữ liệu bị thiếu là 100%). Để làm gọn dữ liệu và để dễ quản lý dataframe hơn, ta sẽ bỏ đi những cột không có ý nghĩa đó

In [7]:
del_cols = []
for i in range(len(missing_ratio)):
    if missing_ratio[i] == 1:
        del_cols.append(missing_ratio.index[i])
df = df.drop(del_cols,axis=1)

## 5. Mỗi cột có ý nghĩa gì?

In [8]:
df.columns

Index(['key', 'class', 'expire_time_gmt', 'obs_id', 'obs_name',
       'valid_time_gmt', 'day_ind', 'temp', 'wx_icon', 'icon_extd',
       'wx_phrase', 'dewPt', 'heat_index', 'rh', 'pressure', 'vis', 'wc',
       'wdir', 'wdir_cardinal', 'gust', 'wspd', 'uv_desc', 'feels_like',
       'uv_index', 'clds'],
      dtype='object')

Dựa trên thông tin tập dữ liệu đã mô tả ở phần Thu thập dữ liệu, ta chọn ra những cột liên quan và cần thiết:
- Thời điểm biểu diễn các chỉ số thời tiết: valid_time_gmt
- Thông tin về chỉ số thời tiết: temp, dewPt, rh, wdir_cardinal, wspd, pressure, vis, wx_phrase ...

Như vậy các cột bị loại bỏ bao gồm: day_ind, wx_icon, icon_extd, heat_index, wc, wdir, gust, uv_desc, feels_like, uv_index, clds ...

### Loại bỏ một số cột không cần thiết

In [9]:
# Huy Trương loại bỏ các cột không cần thiết dựa trên ý nghĩa key, class, expire_time_gmt, obs_id, obs_name,

## 6. Mỗi cột hiện đang có kiểu dữ liệu gì? Có cột nào có kiểu dữ liệu chưa phù hợp để có thể xử lý tiếp hay không?

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17390 entries, 0 to 17389
Data columns (total 25 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   key              17390 non-null  object 
 1   class            17390 non-null  object 
 2   expire_time_gmt  17390 non-null  int64  
 3   obs_id           17390 non-null  object 
 4   obs_name         17390 non-null  object 
 5   valid_time_gmt   17390 non-null  int64  
 6   day_ind          17390 non-null  object 
 7   temp             17390 non-null  int64  
 8   wx_icon          17390 non-null  int64  
 9   icon_extd        17390 non-null  int64  
 10  wx_phrase        17390 non-null  object 
 11  dewPt            17388 non-null  float64
 12  heat_index       17388 non-null  float64
 13  rh               17388 non-null  float64
 14  pressure         17380 non-null  float64
 15  vis              17390 non-null  float64
 16  wc               17390 non-null  int64  
 17  wdir        

In [11]:
pd.set_option('display.max_columns', None)
df.head()

,key,class,expire_time_gmt,obs_id,obs_name,valid_time_gmt,day_ind,temp,wx_icon,icon_extd,wx_phrase,dewPt,heat_index,rh,pressure,vis,wc,wdir,wdir_cardinal,gust,wspd,uv_desc,feels_like,uv_index,clds
0,VVTS,observation,1609441200,VVTS,Ho Chi Minh City,1609434000,N,75,29,2900,Partly Cloudy,61.0,75.0,61.0,29.85,6.0,75,30.0,NNE,NaN,8,Low,75.0,0,SCT
1,VVTS,observation,1609443000,VVTS,Ho Chi Minh City,1609435800,N,75,29,2900,Partly Cloudy,61.0,75.0,61.0,29.85,6.0,75,30.0,NNE,NaN,7,Low,75.0,0,SCT
2,VVTS,observation,1609444800,VVTS,Ho Chi Minh City,1609437600,N,75,29,2900,Partly Cloudy,61.0,75.0,61.0,29.82,6.0,75,40.0,NE,NaN,7,Low,75.0,0,SCT
3,VVTS,observation,1609446600,VVTS,Ho Chi Minh City,1609439400,N,73,29,2900,Partly Cloudy,61.0,73.0,65.0,29.82,4.0,73,40.0,NE,NaN,6,Low,73.0,0,SCT
4,VVTS,observation,1609448400,VVTS,Ho Chi Minh City,1609441200,N,73,29,2900,Partly Cloudy,61.0,73.0,65.0,29.82,4.0,73,20.0,NNE,NaN,6,Low,73.0,0,SCT


Ta có thể thấy, cột `expire_time_gmt` và cột `valid_time_gmt` nên là dữ liệu datetime, nhưng hiện giờ đang có kiểu dữ liệu numeric

Ta nên đưa giá trị 2 cột này về dạng datetime để có thể nhìn nhận và khám phá dễ hơn

In [12]:
def convert_time_column_to_datetime_type(df):
    if df['expire_time_gmt'].dtypes == 'int64':
        df['expire_time_gmt']= df['expire_time_gmt'].apply(datetime.fromtimestamp)
    if df['valid_time_gmt'].dtypes == 'int64':
        df['valid_time_gmt']= df['valid_time_gmt'].apply(datetime.fromtimestamp)
    return df


In [13]:
df = convert_time_column_to_datetime_type(df)

## 7. Với mỗi cột có kiểu dữ liệu số, các giá trị phân bố như thế nào?

Với các cột có kiểu dữ liệu số, ta sẽ tính:
- Tỉ lệ % (từ 0 đến 100) các giá trị thiếu 
- Giá trị min
- Giá trị lower quartile (phân vị 25)
- Giá trị median (phân vị 50)
- Giá trị upper quartile (phân vị 75)
- Giá trị max

Ta sẽ lưu kết quả vào DataFrame `num_col_info_df`, trong đó: 
- Tên của các cột là tên của các cột số trong `df`
- Tên của các dòng là: "missing_ratio", "min", "lower_quartile", "median", "upper_quartile", "max"  

In [14]:
data = {
    "row_name" : ["missing_ratio", "min", "lower_quartile", "median", "upper_quartile", "max"]
}
col_name = []
for i in df.columns:
    if (df[i].dtypes == 'float64') or (df[i].dtypes == 'int64'):
        col_name.append(i)

for i in col_name:
    _data = []
    missing_data = df[i].isna().sum()
    _data.append((missing_data / len(df[i]) * 100).round(1))
    _data.append(df[i].min())
    percentile = df[i].quantile([0.25,0.5,0.75])
    _data.append(percentile[0.25])
    _data.append(percentile[0.5])
    _data.append(percentile[0.75])
    _data.append(df[i].max())
    data[i] = _data
num_col_info_df = pd.DataFrame(data).set_index('row_name')

In [15]:
num_col_info_df

,temp,wx_icon,icon_extd,dewPt,heat_index,rh,pressure,vis,wc,wdir,gust,wspd,feels_like,uv_index
row_name,,,,,,,,,,,,,,
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0,0.10,0.00,0.0,25.6,99.5,0.0,0.0,0.0
min,64.0,4.0,400.0,46.0,64.0,21.0,29.52,0.37,64.0,10.0,14.0,0.0,64.0,0.0
lower_quartile,79.0,28.0,2800.0,72.0,82.0,66.0,29.70,6.00,79.0,130.0,22.0,3.0,82.0,0.0
median,82.0,29.0,2900.0,75.0,88.0,79.0,29.76,6.00,82.0,200.0,26.0,6.0,88.0,0.0
upper_quartile,86.0,30.0,3000.0,77.0,95.0,89.0,29.82,6.00,86.0,250.0,32.0,8.0,95.0,4.0
max,99.0,47.0,3849.0,84.0,114.0,100.0,29.97,6.00,99.0,360.0,45.0,31.0,114.0,15.0


## 8. Với mỗi cột có kiểu dữ liệu không phải dạng số, các giá trị được phân bố như thế nào?

Thực hiện thống kê và lưu vào một dataframe với các dòng là đại diện cho các giá trị như sau:
- Tỉ lệ % (từ 0 đến 100) các giá trị thiếu (missing_ratio).
- Số lượng các giá trị khác nhau (không xét giá trị thiếu) (num_values).
- Tỉ lệ % (từ 0 đến 100) của mỗi giá trị được sort theo tỉ lệ % giảm dần (không xét giá trị thiếu, tỉ lệ là tỉ lệ so với số lượng các giá trị không thiếu): dùng dictionary để lưu, key là giá trị, value là tỉ lệ % (value_ratios).

In [16]:
categorical_df=df.select_dtypes(exclude=['int64','float64','datetime64'])
data = {
    "row_name" : ["missing_ratio", "num_values", "value_ratios"]
}
col_names = list(categorical_df.columns)
for i in col_names:
    _data = []

    #Tính missing ratio
    missing_ratio = (categorical_df[i].isna().sum())/len(df[i]) * 100
    _data.append(missing_ratio)

    #Tính số lượng giá trị khác nhau
    num_values = categorical_df[i].dropna().nunique()
    _data.append(num_values)

    #Tính tỉ lệ của các giá trị
    dict_value_ratios ={}
    values = categorical_df[i].value_counts(normalize=True,sort=True)
    name_values = list(values.index)
    for k in range(len(name_values)):
        dict_value_ratios[name_values[k]] = round(values[k]*100,1)
    _data.append(dict_value_ratios)
    
    data[i] = _data

cat_col_info_df = pd.DataFrame(data).set_index('row_name')


In [17]:
cat_col_info_df

,key,class,obs_id,obs_name,day_ind,wx_phrase,wdir_cardinal,uv_desc,clds
row_name,,,,,,,,,
missing_ratio,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
num_values,1,1,1,1,2,24,18,5,5
value_ratios,{'VVTS': 100.0},{'observation': 100.0},{'VVTS': 100.0},{'Ho Chi Minh City': 100.0},"{'D': 52.6, 'N': 47.4}","{'Partly Cloudy': 48.5, 'Mostly Cloudy': 25.1,...","{'VAR': 23.3, 'WSW': 10.1, 'W': 8.6, 'SE': 8.3...","{'Low': 70.6, 'Moderate': 11.6, 'High': 9.0, '...","{'SCT': 51.5, 'BKN': 29.1, 'FEW': 17.3, 'CLR':..."
